# Task 2: Quantitative Analysis using PyNance and TA-Lib (Multi-Ticker)

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib
from pynance import Pynance
import os
import re # For extracting ticker from filename

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 7)

## 2. Define Data Path and Identify Stock Data Files

In [ ]:
data_directory = '../data/raw/yfinance_data/' # Relative path from notebooks directory
stock_files = {}

if os.path.exists(data_directory):
    for filename in os.listdir(data_directory):
        if filename.endswith('_historical_data.csv'):
            match = re.match(r"(.+)_historical_data.csv", filename)
            if match:
                ticker_symbol = match.group(1)
                stock_files[ticker_symbol] = os.path.join(data_directory, filename)
else:
    print(f"Error: Data directory not found at {os.path.abspath(data_directory)}")

if not stock_files:
    print(f"No stock data files found in {data_directory}. Please check the path and filenames.")
else:
    print(f"Found stock data for the following tickers: {list(stock_files.keys())}")

## 3. Loop Through Each Stock for Analysis and Visualization

In [ ]:
for ticker_symbol, filepath in stock_files.items():
    print(f"\nProcessing data for ticker: {ticker_symbol} from {filepath}...")
    
    # --- 3.1 Load and Prepare Data ---
    try:
        data = pd.read_csv(filepath)
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    except Exception as e:
        print(f"Error loading data for {ticker_symbol}: {e}")
        continue # Skip to the next ticker if data loading fails

    # Verify required columns
    required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        print(f"Error for {ticker_symbol}: Missing required columns: {missing_columns}. Skipping this ticker.")
        continue
    
    print(f"Data loaded successfully for {ticker_symbol}. Shape: {data.shape}")
    # print(data.head())
    # print(data.isnull().sum())

    # --- 3.2 Apply Analysis Indicators with TA-Lib ---
    print(f"Calculating TA-Lib indicators for {ticker_symbol}...")
    try:
        data['SMA_20'] = talib.SMA(data['Close'], timeperiod=20)
        data['EMA_20'] = talib.EMA(data['Close'], timeperiod=20)
        data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
        macd, macdsignal, macdhist = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        data['MACD'] = macd
        data['MACD_Signal'] = macdsignal
        data['MACD_Hist'] = macdhist
    except Exception as e:
        print(f"Error calculating TA-Lib indicators for {ticker_symbol}: {e}")
        # Continue with available data if some indicators fail
        pass

    # --- 3.3 Use PyNance for Financial Metrics ---
    print(f"Attempting PyNance metrics for {ticker_symbol}...")
    pynance_data = data.reset_index().rename(columns={
        'Date': 'date',
        'Open': 'open',
        'High': 'high',
        'Low': 'low',
        'Close': 'close',
        'Adj Close': 'adj_close',
        'Volume': 'volume'
    })
    pynance_data['date'] = pd.to_datetime(pynance_data['date'])
    
    try:
        stock_analyzer = Pynance(pynance_data)
        returns = stock_analyzer.get_returns()
        # print(f"PyNance Returns for {ticker_symbol}:\n{returns.tail()}")
        log_returns = stock_analyzer.get_log_returns()
        # print(f"PyNance Log Returns for {ticker_symbol}:\n{log_returns.tail()}")
    except Exception as e:
        print(f"Error using PyNance for {ticker_symbol}: {e}. Proceeding without PyNance metrics.")

    # --- 3.4 Visualize the Data ---
    print(f"Generating visualizations for {ticker_symbol}...")
    
    # Plot 1: Stock Price with Moving Averages
    if 'SMA_20' in data.columns and 'EMA_20' in data.columns:
        plt.figure(figsize=(15,7))
        plt.plot(data.index, data['Close'], label='Close Price')
        plt.plot(data.index, data['SMA_20'], label='20-Day SMA')
        plt.plot(data.index, data['EMA_20'], label='20-Day EMA')
        plt.title(f'{ticker_symbol} Close Price with Moving Averages')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()
    
    # Plot 2: RSI
    if 'RSI' in data.columns:
        plt.figure(figsize=(15,7))
        plt.plot(data.index, data['RSI'], label='RSI')
        plt.axhline(70, linestyle='--', alpha=0.5, color='red', label='Overbought (70)')
        plt.axhline(30, linestyle='--', alpha=0.5, color='green', label='Oversold (30)')
        plt.title(f'{ticker_symbol} RSI (14-Day)')
        plt.xlabel('Date')
        plt.ylabel('RSI')
        plt.legend()
        plt.show()
    
    # Plot 3: MACD
    if 'MACD' in data.columns and 'MACD_Signal' in data.columns and 'MACD_Hist' in data.columns:
        fig, ax = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
        ax[0].plot(data.index, data['Close'], label='Close Price')
        ax[0].plot(data.index, data['MACD'], label='MACD', color='blue')
        ax[0].plot(data.index, data['MACD_Signal'], label='Signal Line', color='red')
        ax[0].set_title(f'{ticker_symbol} Close Price, MACD & Signal Line')
        ax[0].legend()
        
        ax[1].bar(data.index, data['MACD_Hist'], label='MACD Histogram', color=data['MACD_Hist'].apply(lambda x: 'g' if x >=0 else 'r'))
        ax[1].set_title(f'{ticker_symbol} MACD Histogram')
        ax[1].legend()
        plt.xlabel('Date')
        plt.tight_layout()
        plt.show()
        
    print(f"Finished processing for ticker: {ticker_symbol}")

print("\nAll specified stock data files have been processed.")

## 4. Summary and Next Steps

This notebook demonstrates:
1. Identifying and loading multiple stock data files from a specified directory.
2. For each stock:
    - Calculating key technical indicators using TA-Lib (SMA, EMA, RSI, MACD).
    - Attempting to use PyNance for financial metrics.
    - Visualizing these indicators to understand their impact on stock prices.

Further analysis could involve:
- Deeper exploration of PyNance capabilities if the library proves stable and useful.
- Aggregating results or performing comparative analysis across tickers.
- Integrating sentiment data (from other project tasks) with these quantitative indicators for each stock.
- Building predictive models based on the combined dataset.